# Ex10: スムーズシェーディング

解像度の異なるメッシュファイルを入力にして，<br>
拡散反射（光沢のない反射）と鏡面反射（ハイライト）が
どのように解像度に影響されるか分析してください．

## Pr10-2の実装部分

- lambert_shading, blinn_shadingの実装を転記してください．


In [3]:
# Work: 以下の関数を修正．
def lambert_shading(k_d, I_i, L, N):
    return k_d * I_i *np.dot(L,N)
# Work: 以下の関数を修正．


def blinn_shading(k_s, I_i, H, N, alpha):
    k_s * I_i * (np.dot(H,N)**alpha)


Work: メッシュの解像度と照明計算結果の違いを分析してみましょう

拡散反射，鏡面反射の個別の結果を見てみましょう:
- 拡散反射の効果: I_diffuse
- 鏡面反射の効果: I_specular
- (I_diffuse, I_specular)=(1, 0)の時，拡散反射の効果のみ
- (I_diffuse, I_specular)=(0, 1)の時，鏡面反射の効果のみ

メッシュの解像度による違いを確認してみてください．
- 'suzanne_smooth.obj'，'suzanne_smooth_2.obj'， 'suzanne_smooth_3.obj'の順にメッシュが細かくなります．
- numVerticesに頂点数，numFacesに面数が出ます．
- 拡散反射，鏡面反射はどのような性質があるでしょうか？
- 計算時間は？

In [4]:
import numpy as np
from mesh import *
from transform import *

# @title Shadingパラメータ { run: "auto" }

# @markdown 入力ファイル
mesh_file = 'suzanne_smooth.obj'  # @param {type:"string"}

# @markdown 入射光の強さ
I_diffuse = 1  # @param {type: "slider", min: 0.0, max: 1.0, step: 0.01}
I_specular = 0  # @param {type: "slider", min: 0.0, max: 1.0, step: 0.01}

# @markdown 拡散反射率 k_d (RGB)
k_d_r = 0.2  # @param {type: "slider", min: 0.0, max: 1.0, step: 0.01}
k_d_g = 0.3  # @param {type: "slider", min: 0.0, max: 1.0, step: 0.01}
k_d_b = 1.0  # @param {type: "slider", min: 0.0, max: 1.0, step: 0.01}
k_d = np.array([k_d_r, k_d_g, k_d_b])

# @markdown 鏡面反射率 k_s (RGB)
k_s_r = 1.0  # @param {type: "slider", min: 0.0, max: 1.0, step: 0.01}
k_s_g = 1.0  # @param {type: "slider", min: 0.0, max: 1.0, step: 0.01}
k_s_b = 1.0  # @param {type: "slider", min: 0.0, max: 1.0, step: 0.01}
k_s = np.array([1.0, 1.0, 1.0])

# @markdown ハイライトの鋭さ
alpha = 20  # @param {type: "slider", min: 0, max: 200}

# @markdown 視点パラメータ
elev = 20  # @param {type: "slider", min: 0, max: 90}
azim = 30  # @param {type: "slider", min: 0, max: 360}

mesh = loadOBJ(mesh_file)
mesh.computeFaceNormals()

P = mesh.V
F = mesh.F
N_f = mesh.N_f

print(mesh_file)
print("- numVertices: ", len(P))
print("- numFaces: ", len(F))

# ライトベクトル
L = np.array([0, 0, 1])

# 視線ベクトルの計算
R = rotate_y(azim) @ rotate_x(-elev)
V = np.array([0, 0, 1]) @ R[:3, :3].T

# ハーフベクトルの計算
H = L + V
H /= np.linalg.norm(H)


# 拡散反射の計算
I_d = [lambert_shading(k_d, I_diffuse, L, N) for N in N_f]
I_d = np.clip(I_d, 0, 1)

# 鏡面反射の計算
I_s = [blinn_shading(k_s, I_specular, H, N, alpha) for N in N_f]
I_s = np.clip(I_s, 0, 1)

# 計算結果の組み合わせ
I = I_d + I_s
I = np.clip(I, 0, 1)

fig = plt.figure(figsize=(15, 15))

viewer = MeshViewer(fig)

viewer.addMesh(P, F, I,  edgecolor=I, alpha=1.0, linewidth=1.0)

viewer.setView(elev=elev, azim=azim)
viewer.show()


suzanne_smooth.obj
- numVertices:  7958
- numFaces:  7872


TypeError: '>=' not supported between instances of 'NoneType' and 'int'